In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
import mecab
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [2]:
import os
filelist = os.listdir('../데이터/기업별_뉴스/~0-126(2018-2022)')

In [3]:
filelist[:5]

['DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20230601-191902.xlsx',
 'DeepSearch-CMG제약-news-2019-01-01-2019-12-31-20230601-192349.xlsx',
 'DeepSearch-CMG제약-news-2020-01-01-2020-12-31-20230601-192831.xlsx',
 'DeepSearch-CMG제약-news-2021-01-01-2021-12-31-20230601-193311.xlsx',
 'DeepSearch-CMG제약-news-2022-01-01-2022-12-31-20230601-193756.xlsx']

In [4]:
# for i, df in enumerate(df_list3):
#     for j,title in enumerate(df['report_nm']):
#         if ("단일판매" in title) or ("보고서" in title) or("주주총회" in title) or("참고서류" in title):
#             # print(1)
#             drop_df = drop_df.append({'df':i,'row':j}, ignore_index=True)

In [5]:
# ['2018-01-01-2018-12-31','2019-01-01-2019-12-31']

In [6]:
flie_18 = []
for name in filelist:
#     # print(type(name))
    if  ('2018-01-01-2018-12-31' in name):
        flie_18.append(name)

In [7]:
# flie_1819[-5]

In [8]:
df18 = []
for name in flie_18:
    df = pd.read_excel(f'../데이터/기업별_뉴스/~0-126(2018-2022)/{name}',sheet_name = '#02 Documents')
    df['corp'] = f'{name}'
    df18.append(df)

In [9]:
df18[0]

,date,category,section,publisher,author,title,content_url,attachment,industry.label,industry.score,industry.name,polarity.label,polarity.score,polarity.name,corp
0,2018-12-27,news,economy,전자신문,NaN,[ET투자뉴스]CMG제약_기관의 힘? 대량순매수 이후.. 현재 +3.05%,NaN,NaN,C21,1.000,의료용 물질 및 의약품 제조업,1.0,0.643,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
1,2018-12-24,news,economy,한국경제,NaN,[한경로보뉴스] '와이지엔터테인먼트' 52주 신고가 경신,NaN,NaN,C26,0.590,"전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",1.0,0.667,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
2,2018-12-13,news,economy,한국경제TV,NaN,"CMG제약(058820) 종목알파고 분석, 외국인/기관 실시간 수급과 추가 매수 확률은?",NaN,NaN,C21,1.000,의료용 물질 및 의약품 제조업,1.0,0.941,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
3,2018-12-06,news,economy,한국경제TV,라이온봇,"CMG제약, 전일 대비 약 -4% 하락한 4,515원",NaN,NaN,C21,1.000,의료용 물질 및 의약품 제조업,0.0,0.653,중립,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
4,2018-12-05,news,tech,조선일보,최인준,간암·뇌전증·치매… 한국 신약 10여종 美 상륙한다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018-01-02,news,economy,파이낸셜뉴스,NaN,[fnRASSI]CMG제약(058820) 현재 21.47% ↑,NaN,NaN,Q86,0.703,보건업,1.0,0.998,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
96,2018-01-02,news,economy,서울경제,NaN,"(코)차바이오텍, 상한가 진입.. +30.00% ↑",NaN,NaN,C21,1.000,의료용 물질 및 의약품 제조업,1.0,0.996,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
97,2018-01-02,news,economy,조선비즈,박현익,"[마켓뷰] 국내 증시, 연초 기대감에 상승…코스닥 800선 회복",NaN,NaN,100,0.845,"분류 제외, 기타",NaN,NaN,NaN,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
98,2018-01-02,news,economy,파이낸셜뉴스,NaN,[fnRASSI]CMG제약 전일대비 9.13% 상승,NaN,NaN,100,0.982,"분류 제외, 기타",1.0,0.993,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...


In [10]:
len(df18)

123

In [11]:
import os
filelist = os.listdir('../데이터/기업별_뉴스/0-50(2010-2017)')

In [12]:
filelist[192]

'DeepSearch-디아이-news-2016-01-01-2016-12-31-20230605-151152.xlsx'

In [13]:
# len(df0_50_1017)

In [14]:
#02 Documents
#02 Documents

In [15]:
df0_50_1017 = []
for name in filelist:
    df = pd.read_excel(f'../데이터/기업별_뉴스/0-50(2010-2017)/{name}',sheet_name = '#02 Documents')
    df['corp'] = f'{name}'
    df0_50_1017.append(df)

In [16]:
import os
filelist = os.listdir('../데이터/기업별_뉴스/51-99(2010-2017)')

In [17]:
df51_99_1017 = []
for name in filelist:
    df = pd.read_excel(f'../데이터/기업별_뉴스/51-99(2010-2017)/{name}',sheet_name = '#02 Documents')
    df['corp'] = f'{name}'
    df51_99_1017.append(df)

In [18]:
import os
filelist = os.listdir('../데이터/기업별_뉴스/100-126(2010-2017)')

In [19]:
df100_126_1017 = []
for name in filelist:
    df = pd.read_excel(f'../데이터/기업별_뉴스/100-126(2010-2017)/{name}',sheet_name = '#02 Documents')
    df['corp'] = f'{name}'
    df100_126_1017.append(df)

In [20]:
df = df18 + df0_50_1017 + df51_99_1017 + df100_126_1017

In [21]:
len(df100_126_1017)

125

In [22]:
df[0].head(2)

,date,category,section,publisher,author,title,content_url,attachment,industry.label,industry.score,industry.name,polarity.label,polarity.score,polarity.name,corp
0,2018-12-27,news,economy,전자신문,NaN,[ET투자뉴스]CMG제약_기관의 힘? 대량순매수 이후.. 현재 +3.05%,NaN,NaN,C21,1.00,의료용 물질 및 의약품 제조업,1.0,0.643,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...
1,2018-12-24,news,economy,한국경제,NaN,[한경로보뉴스] '와이지엔터테인먼트' 52주 신고가 경신,NaN,NaN,C26,0.59,"전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",1.0,0.667,긍정,DeepSearch-CMG제약-news-2018-01-01-2018-12-31-20...


In [23]:
len(df)

931

In [24]:
total = pd.concat(df)

In [25]:
# len(total)

In [26]:
total.to_csv('./df10-18.csv',index=False)

# 토크나이즈

In [27]:
total = total.iloc[:,[5,11]]

In [28]:
total.head(2)

,title,polarity.label
0,[ET투자뉴스]CMG제약_기관의 힘? 대량순매수 이후.. 현재 +3.05%,1.0
1,[한경로보뉴스] '와이지엔터테인먼트' 52주 신고가 경신,1.0


In [29]:
# total['polarity.label'] = total['polarity.label'].replace([0,1,-1],[0,1,2])